# Import & Configuration

In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torch.autograd import grad
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import warnings
# select multiline -> 'ctrl + /' -> multiline comment
# print option
torch.set_printoptions(precision=4, linewidth=50000, sci_mode=None)
# Control Warning Message
warnings.filterwarnings(action='ignore')
writer = SummaryWriter('runs')

# Preparing Data

In [2]:
dataset_dir = '~/dataset'
train_data = datasets.MNIST(root=dataset_dir, train=True,  download=True, transform=ToTensor())
test_data  = datasets.MNIST(root=dataset_dir, train=False, download=True, transform=ToTensor())

# Data Loader

In [3]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data,  batch_size=batch_size)

# Model Build-up

In [4]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print("Using {} device".format(device))

class NN(nn.Module):
	def __init__(self):
		super(NN, self).__init__()
		self.flatten = nn.Flatten()
		self.fc1 = nn.Linear(28*28, 16)
		self.fc2 = nn.Linear(16, 16)
		self.fc3 = nn.Linear(16, 10)

	def forward(self, x):
		x = self.flatten(x)
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		logits = self.fc3(x)
		return logits
	
model = NN().to(device)
print(model)

Using cpu device
NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)


# 훈련(Training) 전 모델 사용

In [5]:
train_dataloader.dataset.data[3]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 124, 253, 255,  63,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  96, 2

In [6]:
images, labels = next(iter(train_dataloader))
images.size()

torch.Size([64, 1, 28, 28])

In [7]:
a_input = images[0]
a_input.size()

torch.Size([1, 28, 28])

In [8]:
print(model.flatten(images).size())
print(model.fc1.weight.size())

torch.Size([64, 784])
torch.Size([16, 784])


In [9]:
model.fc1(model.flatten(a_input))

tensor([[ 0.1085,  0.0798, -0.2707,  0.0119, -0.0894,  0.0932, -0.0477,  0.1152, -0.2524, -0.3236,  0.0881, -0.4643,  0.2018, -0.0513,  0.3254, -0.0665]], grad_fn=<AddmmBackward>)

In [10]:
model.flatten(a_input).size()

torch.Size([1, 784])

In [11]:
a_input.view(-1, 28*28).size()

torch.Size([1, 784])

In [12]:
model.fc1.weight.size()

torch.Size([16, 784])

In [13]:
model.fc1.bias.size()

torch.Size([16])

In [14]:
layer_fc1 = torch.matmul(model.flatten(a_input), model.fc1.weight.t()) + model.fc1.bias
print(layer_fc1)
torch.equal(layer_fc1, model.fc1(model.flatten(a_input)))

tensor([[ 0.1085,  0.0798, -0.2707,  0.0119, -0.0894,  0.0932, -0.0477,  0.1152, -0.2524, -0.3236,  0.0881, -0.4643,  0.2018, -0.0513,  0.3254, -0.0665]], grad_fn=<AddBackward0>)


True

In [15]:
layer_fc1 = torch.matmul(a_input.view(-1, 28*28), model.fc1.weight.t()) + model.fc1.bias
print(layer_fc1)
torch.equal(layer_fc1, model.fc1(model.flatten(a_input)))

tensor([[ 0.1085,  0.0798, -0.2707,  0.0119, -0.0894,  0.0932, -0.0477,  0.1152, -0.2524, -0.3236,  0.0881, -0.4643,  0.2018, -0.0513,  0.3254, -0.0665]], grad_fn=<AddBackward0>)


True

In [16]:
layer_fc1 = torch.matmul(a_input.view(-1, 28*28), model.fc1.weight.t()) + model.fc1.bias
layer_fc2 = torch.matmul(F.relu(layer_fc1), model.fc2.weight.t()) + model.fc2.bias
layer_fc3 = torch.matmul(F.relu(layer_fc2), model.fc3.weight.t()) + model.fc3.bias

In [17]:
model(a_input)

tensor([[-0.2329,  0.0464,  0.0022,  0.1612,  0.0521, -0.1067, -0.2053, -0.0629, -0.1396,  0.0975]], grad_fn=<AddmmBackward>)

In [18]:
layer_fc3

tensor([[-0.2329,  0.0464,  0.0022,  0.1612,  0.0521, -0.1067, -0.2053, -0.0629, -0.1396,  0.0975]], grad_fn=<AddBackward0>)

In [19]:
layer_fc3.size()

torch.Size([1, 10])

In [20]:
torch.equal(layer_fc3, model(a_input))

True

# Define Loss Function & Optimizer

In [21]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
train_loss = 0
test_loss = 0

In [22]:
labels[0]

tensor(5)

In [23]:
model(images).size()

torch.Size([64, 10])

In [24]:
labels.size()

torch.Size([64])

In [25]:
loss_func(model(images), labels)

tensor(2.2935, grad_fn=<NllLossBackward>)

In [26]:
layer_fc3.size()

torch.Size([1, 10])

In [27]:
labels[0].size()

torch.Size([])

In [28]:
labels[0].view(1).size()

torch.Size([1])

In [29]:
loss_func(layer_fc3, labels[0].view(1))

tensor(2.3783, grad_fn=<NllLossBackward>)

In [30]:
def myCrossEntropyLoss(logits, labels):
	cee = 0
	for idx, logit in enumerate(logits):
		y  = labels[idx]
		y_ = F.softmax(logit)[y]
		cee = cee - torch.log(y_)/len(logits)
	print(f'Loss: {cee}')
	return cee

In [31]:
cee = myCrossEntropyLoss(layer_fc3, labels)
torch.equal(cee, loss_func(layer_fc3, labels[0].view(1)))

Loss: 2.378310203552246


True

In [32]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
optimizer.zero_grad()

In [33]:
pred = model(images[0])
loss = loss_func(pred, labels[0].view(1))

In [34]:
model.fc3.weight.grad

In [35]:
loss.backward()

In [36]:
model.fc3.weight.grad

tensor([[ 0.0000,  0.0169,  0.0053,  0.0000,  0.0063,  0.0000,  0.0025,  0.0070,  0.0048,  0.0056,  0.0000,  0.0223,  0.0132,  0.0000,  0.0000,  0.0102],
        [ 0.0000,  0.0223,  0.0070,  0.0000,  0.0083,  0.0000,  0.0034,  0.0093,  0.0063,  0.0074,  0.0000,  0.0294,  0.0174,  0.0000,  0.0000,  0.0135],
        [ 0.0000,  0.0214,  0.0067,  0.0000,  0.0080,  0.0000,  0.0032,  0.0089,  0.0060,  0.0071,  0.0000,  0.0282,  0.0167,  0.0000,  0.0000,  0.0129],
        [ 0.0000,  0.0250,  0.0079,  0.0000,  0.0093,  0.0000,  0.0038,  0.0104,  0.0071,  0.0083,  0.0000,  0.0330,  0.0195,  0.0000,  0.0000,  0.0151],
        [ 0.0000,  0.0225,  0.0071,  0.0000,  0.0084,  0.0000,  0.0034,  0.0094,  0.0064,  0.0074,  0.0000,  0.0296,  0.0175,  0.0000,  0.0000,  0.0136],
        [-0.0000, -0.1875, -0.0589, -0.0000, -0.0700, -0.0000, -0.0282, -0.0781, -0.0531, -0.0620, -0.0000, -0.2471, -0.1462, -0.0000, -0.0000, -0.1133],
        [ 0.0000,  0.0174,  0.0055,  0.0000,  0.0065,  0.0000,  0.0026,  0.0

In [37]:
before_training = model.fc3.weight.clone().detach()
before_training

tensor([[ 0.1750, -0.1287, -0.0902,  0.2107, -0.0483, -0.0139, -0.1286,  0.2436,  0.0267,  0.0610,  0.1243,  0.0817, -0.2035, -0.2417, -0.1590,  0.0428],
        [ 0.1264,  0.0822,  0.2236, -0.0562,  0.0035,  0.0723, -0.2063,  0.0979,  0.1763, -0.1489,  0.1637,  0.2431, -0.1810,  0.1353,  0.2021,  0.0085],
        [-0.0924,  0.0389, -0.0249,  0.1632, -0.2444,  0.1350, -0.2495, -0.1282, -0.0348,  0.0310,  0.1813, -0.2181, -0.1959, -0.1994, -0.0335,  0.0327],
        [-0.2007,  0.1495, -0.2259,  0.0909, -0.0700,  0.0561, -0.0032,  0.0201, -0.0749, -0.2481, -0.1285,  0.2216,  0.2127,  0.0663,  0.1453, -0.1051],
        [-0.0071, -0.2253, -0.0555, -0.2311,  0.0015, -0.1449,  0.2466, -0.0707,  0.0715,  0.2438, -0.0492,  0.0046,  0.0418, -0.0353,  0.1927,  0.1976],
        [ 0.0549, -0.2247, -0.2277, -0.0884, -0.1646, -0.1301, -0.1878,  0.1994,  0.1693,  0.1127, -0.1307, -0.2377,  0.0484,  0.1716,  0.1420, -0.0022],
        [-0.1298, -0.0939,  0.0607,  0.2299, -0.1827, -0.0609,  0.2212,  0.1

In [38]:
optimizer.step()

In [39]:
model.fc3.weight

Parameter containing:
tensor([[ 0.1750, -0.1289, -0.0902,  0.2107, -0.0484, -0.0139, -0.1287,  0.2436,  0.0267,  0.0609,  0.1243,  0.0815, -0.2036, -0.2417, -0.1590,  0.0427],
        [ 0.1264,  0.0820,  0.2235, -0.0562,  0.0034,  0.0723, -0.2064,  0.0978,  0.1763, -0.1490,  0.1637,  0.2428, -0.1812,  0.1353,  0.2021,  0.0084],
        [-0.0924,  0.0387, -0.0250,  0.1632, -0.2445,  0.1350, -0.2496, -0.1283, -0.0349,  0.0310,  0.1813, -0.2184, -0.1961, -0.1994, -0.0335,  0.0326],
        [-0.2007,  0.1493, -0.2260,  0.0909, -0.0701,  0.0561, -0.0032,  0.0200, -0.0750, -0.2482, -0.1285,  0.2213,  0.2125,  0.0663,  0.1453, -0.1052],
        [-0.0071, -0.2255, -0.0556, -0.2311,  0.0014, -0.1449,  0.2466, -0.0708,  0.0715,  0.2437, -0.0492,  0.0043,  0.0416, -0.0353,  0.1927,  0.1975],
        [ 0.0549, -0.2228, -0.2271, -0.0884, -0.1639, -0.1301, -0.1875,  0.2001,  0.1699,  0.1134, -0.1307, -0.2352,  0.0499,  0.1716,  0.1420, -0.0011],
        [-0.1298, -0.0941,  0.0607,  0.2299, -0.1827, 

In [40]:
torch.equal(before_training[0][0], model.fc3.weight[0][0])

True

In [41]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[-0.0297, -0.0256,  0.0351,  ..., -0.0076, -0.0124,  0.0043],
           [ 0.0241, -0.0155,  0.0266,  ..., -0.0235,  0.0289,  0.0357],
           [ 0.0023,  0.0281,  0.0080,  ...,  0.0141, -0.0080,  0.0247],
           ...,
           [ 0.0027, -0.0152, -0.0005,  ..., -0.0141,  0.0264,  0.0188],
           [ 0.0180,  0.0305,  0.0340,  ..., -0.0047, -0.0077,  0.0248],
           [ 0.0329,  0.0333,  0.0189,  ..., -0.0354,  0.0121, -0.0170]], requires_grad=True),
   Parameter containing:
   tensor([ 0.0348,  0.0227,  0.0091,  0.0110, -0.0347,  0.0225,  0.0286,  0.0071,  0.0270,  0.0123,  0.0253,  0.0131,  0.0258,  0.0003,  0.0167,  0.0351], requires_grad=True),
   Parameter containing:
   tensor([[ 0.2184, -0.1006,  0.1165, -0.2377,  0.0142, -0.2131, -0.2422,  0.2175,  0.0339, -0.2054, -0.1353, -0.1485, -0.2431,  0.0934, -0.0510, -0.0994],
           [ 0.1819,  0.1668, -0.1939, -0.2437, -0.0299,  0.1234,  0.0464,  0.1891, -0.0769, -0.0777, -0.

In [42]:
optimizer.param_groups[0]['lr']

0.01

In [43]:
torch.equal(before_training[0][0] - optimizer.param_groups[0]['lr'] * model.fc3.weight.grad[0][0] , model.fc3.weight[0][0])

True

In [44]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
optimizer.zero_grad()

In [45]:
pred = model(images)
loss = loss_func(pred, labels)

In [46]:
images[0]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.

In [47]:
loss.backward()

In [48]:
model.fc3.weight.grad

tensor([[ 0.0000e+00, -3.0644e-03,  3.8743e-03,  2.6230e-04, -3.1600e-03,  0.0000e+00, -3.7481e-03,  2.6523e-03, -1.6785e-03, -1.7007e-03,  0.0000e+00, -1.4364e-02, -2.9594e-03, -2.2354e-05,  0.0000e+00, -1.1005e-03],
        [ 0.0000e+00, -6.5290e-03,  1.7637e-04, -2.0299e-04, -3.3200e-03,  0.0000e+00,  2.5607e-05, -3.0064e-03,  3.9526e-04, -1.4833e-03,  0.0000e+00, -1.9085e-03, -3.4982e-03,  1.5408e-04,  0.0000e+00, -1.5963e-03],
        [ 0.0000e+00,  5.9107e-03,  1.4837e-03,  3.3003e-04, -1.8078e-03,  0.0000e+00,  1.7502e-03,  5.8350e-03, -7.7737e-06,  5.9028e-03,  0.0000e+00,  1.1762e-02,  5.8528e-03, -8.5891e-04,  0.0000e+00,  5.1828e-03],
        [ 0.0000e+00, -2.9406e-03, -1.7590e-03,  2.8957e-04, -8.5970e-04,  0.0000e+00, -1.5932e-04,  1.7608e-03, -1.9589e-03,  2.0207e-04,  0.0000e+00, -3.6920e-03,  2.1640e-04,  1.6099e-04,  0.0000e+00,  3.8480e-05],
        [ 0.0000e+00,  2.5440e-03, -1.6979e-03, -7.9597e-04,  1.7274e-03,  0.0000e+00, -3.4775e-03, -2.7498e-03,  2.1088e-03, -3

In [49]:
def train(train_dataloader, model, loss_func, optimizer, epoch, train_loss):
	for batch_index, (img, lab) in enumerate(train_dataloader):
		img, lab = img.to(device), lab.to(device)
		pred = model(img)
		loss = loss_func(pred, lab)
		print(batch_index, end = ' ')
# 		optimizer.zero_grad()
		loss.backward()
		
		optimizer.step()
		
		train_loss += loss.item()
	writer.add_scalar('trainLoss', train_loss/len(train_dataloader.dataset), epoch)
	trainLoss = 0
	return loss

In [50]:
def test(test_dataloader, model, epoch, test_loss):
	model.eval()
	test_loss, correct = 0, 0
	with torch.no_grad():
		for img, lab in test_dataloader:
			img, lab = img.to(device), lab.to(device)
			pred = model(img)
			test_loss += loss_func(pred, lab).item()
			correct += (pred.argmax(1) == lab).type(torch.float).sum().item()
	test_loss /= len(test_dataloader.dataset)
	correct /= len(test_dataloader.dataset)
	print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
	writer.add_scalar('testLoss', test_loss, epoch)
	test_loss = 0

In [51]:
model.state_dict().keys()

odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])

In [52]:
epochs = 5
for epoch in range(epochs):
	print(f"Epoch {epoch+1:3d}: ", end='')
	loss = train(train_dataloader, model, loss_func, optimizer, epoch, train_loss)
	test(test_dataloader, model, epoch, test_loss)
print("Done!")
writer.close()

Epoch   1: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274

In [53]:
model.fc1.weight

Parameter containing:
tensor([[-0.0297, -0.0256,  0.0351,  ..., -0.0076, -0.0124,  0.0043],
        [ 0.0241, -0.0155,  0.0266,  ..., -0.0235,  0.0289,  0.0357],
        [ 0.0023,  0.0281,  0.0080,  ...,  0.0141, -0.0080,  0.0247],
        ...,
        [ 0.0027, -0.0152, -0.0005,  ..., -0.0141,  0.0264,  0.0188],
        [ 0.0180,  0.0305,  0.0340,  ..., -0.0047, -0.0077,  0.0248],
        [ 0.0329,  0.0333,  0.0189,  ..., -0.0354,  0.0121, -0.0170]], requires_grad=True)

In [54]:
model.fc2.weight.grad

tensor([[ 1.5744e+00,  5.6859e-01,  4.8134e+00, -2.7338e-03,  3.6798e-01,  9.2795e-01,  1.9497e+00, -2.0374e-03,  1.8977e+00, -8.9890e-02, -3.3157e-02,  1.9824e-02,  8.3217e-02, -6.4346e-03, -4.4225e-02,  1.2580e+00],
        [ 2.4415e+00,  1.7940e+00,  1.9261e-01,  1.0768e+00, -6.4617e-02,  1.3890e+00,  5.3226e-01,  1.0674e+00,  2.2285e+00,  2.9046e-01,  1.4834e+00,  1.1693e+00,  2.0378e+00, -3.1069e-01,  8.9808e-01,  3.7648e+00],
        [ 1.2804e+00,  1.8796e+00,  1.6328e+00,  7.5941e-02,  3.6955e-01,  8.9219e-01,  6.0784e-01, -3.6053e-01,  1.6417e+00,  5.4127e-01,  3.8321e-02,  8.9872e-01,  6.6316e-01,  5.2462e-01,  8.6481e-01,  4.7609e-01],
        [ 1.5760e-02,  1.2095e-01,  1.2316e-01,  6.7805e-03,  2.0561e-01,  1.2125e-01,  6.3861e-03,  1.0159e-02,  6.1074e-03,  7.2756e-03,  8.9574e-03,  2.0344e-03,  3.9869e-03,  1.4419e-02,  1.0611e-02,  1.3823e-01],
        [-1.4326e-01,  8.1639e-01,  6.3954e+00,  6.7654e-01,  1.2040e+00,  7.7855e-01,  6.3737e+00, -3.9706e-01,  7.0889e-01,  6